In [168]:
import os,sys,time
import random
import numpy as np
import pandas as pd
import torch
import torch.optim as optim
import torch.autograd
from torch.autograd import Variable
from torch import nn,LongTensor,FloatTensor
import torch.utils.data as data
from tqdm import tqdm
from collections import Counter
from scipy.spatial.distance import cdist
import datetime
import torch.nn.functional as Fconda

import time

In [115]:
MIN_INTER             = 5
CUDA                  = False
BATCH_SIZE            = 512
SAMPLES_PER_ITERATION_TR = 100000
SAMPLES_PER_ITERATION_TE = 10000

In [183]:
class DataStore():
  def __init__(self,fn):

    #self.interactions = []
    self.max_u = 1
    self.max_i = 1
    self.user_inter     = {}
    self.tot_items      = set()
    self.tot_items_test = set()
    self.train          = {}
    self.train_inter    = []
    self.test           = {}
    self.valuemap       = {}
    self.load_items(fn)

  def load_items(self,fn):
    
    self.df = pd.read_csv(fn)

    print("Raw interactions: ",len(self.df))
    print("Raw user count: ", len(set(self.df['user'].tolist())) )
    
    filtered = self.df['user'].value_counts()
    filtered = filtered[filtered>=MIN_INTER]
    

    self.df = self.df[self.df['user'].isin(filtered.index)]
    print("Filtered interactions: ", len(self.df))
    print("Filtered user count: ",   len(set(self.df['user'].tolist())) )

    self.max_u = int(self.df['user'].max())
    self.max_i = int(self.df['item'].max())

    self.unique_items = self.df['item'].drop_duplicates()

    self.test  = self.df.groupby('user', group_keys=False).apply(lambda df: df.sample(1))
    self.train = self.df.drop(self.test.index)
    
    print("start init dict")
    self.uidict = {}
    self.iudict = {}

    dfUser = list(self.train["user"])
    dfItem = list(self.train["item"])

    for i in range(len(dfUser)):
        if dfUser[i] in self.uidict:
            self.uidict[dfUser[i]].append(dfItem[i])
        else:
            self.uidict[dfUser[i]] = [dfItem[i]]
            
    for i in range(len(dfItem)):
        if dfItem[i] in self.iudict:
            self.iudict[dfItem[i]].append(dfUser[i])
        else:
            self.iudict[dfItem[i]] = [dfUser[i]]
    print("initialized")

  def sample(self,u):

    while True:
      rand_row = self.train.sample(1)
      if int(rand_row['user']) != u:
        break

    return rand_row

  def batch_sample(self):
    batch = []

    testpos = self.train.sample(SAMPLES_PER_ITERATION_TR)
    #testneg = self.unique_items.sample(SAMPLES_PER_ITERATION_TR, replace=True)
    testneg = self.train.sample(SAMPLES_PER_ITERATION_TR)

    for ind in range(0,len(testpos)):

      user = int(testpos.iloc[ind]["user"])
      pos  = int(testpos.iloc[ind]["item"])
      #neg  = int(testneg.iloc[ind])
      neg  = int(testneg.iloc[ind]["item"])
    
      while neg in self.uidict[user]:
        #neg = int(self.unique_items.sample(1).iloc[0])
        rand = np.random.randint(len(self.train))
        neg = int(self.train.iloc[rand]["item"])
        
      batch.append([user,pos,neg])

    batch = FloatTensor(batch)

    if CUDA:
      batch = batch.cuda()

    return torch.utils.data.DataLoader(batch, batch_size=BATCH_SIZE, shuffle=True)

  def dummy_rank(self):
    batch = []

    testpos = self.test.sample(SAMPLES_PER_ITERATION_TE)
    testneg = self.df.sample(SAMPLES_PER_ITERATION_TE)

    for ind in range(0,len(testpos)):

      user = int(testpos.iloc[ind]["user"])
      pos  = int(testpos.iloc[ind]["item"])
      neg  = int(testneg.iloc[ind]["item"])
    
      while neg in self.uidict[user]:
        neg = int(self.df.sample(1).iloc[0]["item"])
      
      batch.append([user,pos,neg])

    batch = FloatTensor(batch)

    if CUDA:
      batch = batch.cuda()

    pos_items = batch[:,1]
    neg_items = batch[:,2]
    
    print(pos_items)

    auc = 0.0
    for i in range(0,len(pos_items)):
      sp = len(self.iudict[pos_items[i]])
      sn = len(self.iudict[neg_items[i]])

      if sp > sn:
        auc += 1.0
      elif sp==sn:
        auc += 0.5

    return auc / len(testpos)

  '''
  def dummy_rank(self):
    batch = []

    testpos = self.test.sample(SAMPLES_PER_ITERATION_TE)
    testneg = self.unique_items.sample(SAMPLES_PER_ITERATION_TE)

    for ind in range(0,len(testpos)):

      user = int(testpos.iloc[ind]["user"])
      pos  = int(testpos.iloc[ind]["item"])
      neg  = int(testneg.iloc[ind])
    
      while neg in self.uidict[user]:
        neg = int(self.unique_items.sample(1).iloc[0])
      
      batch.append([user,pos,neg])

    batch = FloatTensor(batch)

    if CUDA:
      batch = batch.cuda()

    pos_items = batch[:,1])
    neg_items = batch[:,2])
    
    print(pos_items)

    auc = 0.0
    for i in range(0,len(pos_items)):
      sp = len(self.uidict[pos_items[i]])
      sn = len(self.uidict[neg_items[i]])

      if sp > sn:
        auc += 1.0
      elif sp==sn:
        auc += 0.5

    return auc / len(testpos)
  '''

In [192]:
class BPR(nn.Module):
  def __init__(self,datastore):
    super(BPR, self).__init__()
    self.logs      = False

    self.bias      = True

    self.K         = 40
    self.user_embs = nn.Embedding(datastore.max_u+1, self.K)
    self.item_embs = nn.Embedding(datastore.max_u+1, self.K)
    self.item_bias = nn.Embedding(datastore.max_u+1, 1)

    if CUDA:
      self.user_embs = self.user_embs.cuda()
      self.item_embs = self.item_embs.cuda()
      self.item_bias = nn.Embedding(datastore.max_u+1, 1).cuda()

    self.n_iter    = 1000
    self.optimizer = optim.SGD(self.parameters(), lr=0.05, weight_decay=0.01)
    self.datastore = datastore

  def forward(self,user,item):
    preds  = (self.user_embs(user) * self.item_embs(item)).sum(1)

    if self.bias:
      preds += self.item_bias(item)

    return preds

  def bpr_loss(self,pos_preds,neg_preds):
    sig = nn.Sigmoid()
    return (1.0 - sig(pos_preds - neg_preds)).pow(2).sum()

  def auc_test(self):
    batch = []

    testpos = self.datastore.test.sample(SAMPLES_PER_ITERATION_TE)
    #testneg = self.datastore.unique_items.sample(SAMPLES_PER_ITERATION_TE)
    testneg = self.datastore.df.sample(SAMPLES_PER_ITERATION_TE)

    for ind in range(0,len(testpos)):

      user = int(testpos.iloc[ind]["user"])
      pos  = int(testpos.iloc[ind]["item"])
      #neg  = int(testneg.iloc[ind])
      neg  = int(testneg.iloc[ind]["item"])
    
      while neg in self.datastore.uidict[user]:
        #neg = int(self.datastore.unique_items.sample(1).iloc[0])
        rand = np.random.randint(len(self.datastore.df))
        neg = int(self.datastore.df.iloc[rand]["item"])
      
      batch.append([user,pos,neg])
     

    batch = FloatTensor(batch)

    if CUDA:
      batch = batch.cuda()

    users     = Variable(batch[:,0]).long()
    pos_items = Variable(batch[:,1]).long()
    neg_items = Variable(batch[:,2]).long()

    pos_preds = self(users,pos_items)
    neg_preds = self(users,neg_items)

    auc = 0.0
    for i in range(0,len(pos_preds)):
      sp = pos_preds[i]
      sn = neg_preds[i]

      sp = sp.data.cpu().numpy()[0]
      sn = sn.data.cpu().numpy()[0]

      if sp > sn:
        auc += 1.0
      elif sp==sn:
        auc += 0.5

    return auc / len(testpos)

  def train(self):

    for epoch in range(self.n_iter):
      print("epoch: ",epoch)
      running_loss = 0.0
      batch_sample = self.datastore.batch_sample()
      for data in tqdm(batch_sample):

        # zero the parameter gradients
        self.optimizer.zero_grad()

        # forward + backward + optimize

        users     = Variable(data[:,0]).long()
        pos_items = Variable(data[:,1]).long()
        neg_items = Variable(data[:,2]).long()

        pos_preds = self(users,pos_items)
        neg_preds = self(users,neg_items)

        loss = self.bpr_loss(pos_preds,neg_preds)
        loss.backward()

        self.optimizer.step()

        # print statistics
        running_loss += loss.data[0]

      if epoch%10==0 and epoch>0:
        print('AUC test: ', self.auc_test())

In [ ]:
ds = DataStore("data/interactions.csv")

In [190]:
ds.dummy_rank()


  7698
  2206
   184
  ⋮   
  3674
  3862
   199
[torch.FloatTensor of size 10000]



0.50235

In [194]:
bpr = BPR(ds)
bpr.train()

epoch:  0


100%|██████████| 196/196 [00:22<00:00,  8.54it/s]


epoch:  1


100%|██████████| 196/196 [00:21<00:00,  9.06it/s]


epoch:  2


...

100%|██████████| 196/196 [00:20<00:00, 10.57it/s]


epoch:  687


100%|██████████| 196/196 [00:17<00:00, 12.65it/s]


epoch:  688


KeyboardInterrupt: 